# WallStreetBets Market Correlation Analysis

Statistical analysis examining the relationship between r/WallStreetBets discussion patterns and stock market behavior.

## Research Questions
1. Is the discussion on r/WallStreetBets consistent over time?
2. Does r/WallStreetBets mention count correlate with trading volumes?
3. Does r/WallStreetBets mention count correlate with absolute returns?
4. Does sentiment on r/WallStreetBets correlate with percentage price changes?
5. Does sentiment tend to lead or lag stock price changes?

In [ ]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Import our custom functions
from utils import load_ticker_data, clean_data_yfinance_WSB
from analysis import *
from config import STOCK_TICKERS, BONFERRONI_THRESHOLD

print("Setup complete!")
print(f"Analyzing tickers: {STOCK_TICKERS}")
print(f"Bonferroni threshold: {BONFERRONI_THRESHOLD}")

In [ ]:
# Load data
ticker_data = load_ticker_data()

# Verify data loading
print("Available tickers:", list(ticker_data.keys()))
if 'NVDA' in ticker_data:
    print("Sample data shape (NVDA):", ticker_data['NVDA'].shape)
    print("Sample data columns:", list(ticker_data['NVDA'].columns))
    print("\nFirst few rows of NVDA data:")
    print(ticker_data['NVDA'].head())
else:
    print("NVDA data not found")

In [ ]:
# Calculate percentage changes and clean data
for ticker in ticker_data:
    ticker_data[ticker] = calculate_pct_change_close(ticker_data[ticker])
    # Remove first row as we cannot calculate change for first day
    ticker_data[ticker] = ticker_data[ticker].dropna(subset=['Pct_Change_Close'])

print("Data processing complete!")
for ticker, df in ticker_data.items():
    print(f"{ticker}: {len(df)} rows")

## Research Question 1: Discussion Consistency Over Time

**Hypothesis:** WSB discussion is inconsistent over time with dramatic spikes and valleys.

In [ ]:
# Plot mentions over time
ax = plot_mentions(ticker_data)
plt.show()

print("\nConclusion: WSB discussion shows high volatility with dramatic spikes and valleys.")
print("Different tickers peak during different periods, rejecting null hypothesis of consistent discussion.")

## Research Questions 2-4: Correlation Analysis

Testing correlations between:
- WSB mentions vs trading volume
- WSB mentions vs absolute price changes  
- WSB sentiment vs price direction

In [ ]:
# Run all correlation analyses
results = run_all_correlations(ticker_data)

# Display results in a nice table
results_df = pd.DataFrame(results).T
print("\nCorrelation Results:")
print("=" * 60)
print(f"{'Ticker':<8} {'Vol_r':<8} {'Vol_p':<8} {'Abs_r':<8} {'Abs_p':<8} {'Sent_r':<8} {'Sent_p':<8}")
print("-" * 60)

for ticker in results:
    r = results[ticker]
    print(f"{ticker:<8} {r['mention_volume_r']:<8.3f} {r['mention_volume_p']:<8.3f} {r['mention_abs_price_r']:<8.3f} {r['mention_abs_price_p']:<8.3f} {r['sentiment_price_r']:<8.3f} {r['sentiment_price_p']:<8.3f}")

print(f"\nBonferroni threshold: {BONFERRONI_THRESHOLD:.4f}")

In [ ]:
# Research Question 2: Mentions vs Volume
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ticker in enumerate(STOCK_TICKERS):
    if ticker in ticker_data:
        analyze_mention_volume_correlation(ticker_data[ticker], axes[i], ticker)

plt.suptitle('Research Question 2: WSB Mentions vs Trading Volume')
plt.tight_layout()
plt.show()

print("\nConclusion Q2: All tickers show significant positive correlations between mentions and volume.")
print("WSB discussion reliably coincides with elevated trading activity.")

In [ ]:
# Research Question 3: Mentions vs Absolute Price Changes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ticker in enumerate(STOCK_TICKERS):
    if ticker in ticker_data:
        analyze_mention_abs_price_correlation(ticker_data[ticker], axes[i], ticker)

plt.suptitle('Research Question 3: WSB Mentions vs Absolute Price Changes')
plt.tight_layout()
plt.show()

print("\nConclusion Q3: All tickers show positive correlations between mentions and price volatility.")
print("WSB attention consistently accompanies larger price movements regardless of direction.")

In [ ]:
# Research Question 4: Sentiment vs Price Direction
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ticker in enumerate(STOCK_TICKERS):
    if ticker in ticker_data:
        analyze_sentiment_price_correlation(ticker_data[ticker], axes[i], ticker)

plt.suptitle('Research Question 4: WSB Sentiment vs Price Direction')
plt.tight_layout()
plt.show()

print("\nConclusion Q4: Sentiment shows minimal predictive value for price direction.")
print("Even significant correlations explain <1.5% of price variance.")

## Research Question 5: Lead/Lag Analysis

Does WSB sentiment lead or lag stock price movements?

In [ ]:
# Create shifted data for lead/lag analysis
ticker_data_0 = ticker_data.copy()
for ticker, df in ticker_data.items():
    ticker_data_0[ticker] = df.copy().dropna()

# Data relating past sentiment data to current stock prices
ticker_data_lead_1 = create_shifted_data(ticker_data, 1)
ticker_data_lead_2 = create_shifted_data(ticker_data, 2)
ticker_data_lead_3 = create_shifted_data(ticker_data, 3)

# Data relating future sentiment data to current stock prices
ticker_data_lag_1 = create_shifted_data(ticker_data, -1)
ticker_data_lag_2 = create_shifted_data(ticker_data, -2)
ticker_data_lag_3 = create_shifted_data(ticker_data, -3)

lead_lag_data = {
    'lead_3': ticker_data_lead_3,
    'lead_2': ticker_data_lead_2,
    'lead_1': ticker_data_lead_1,
    '0': ticker_data_0,
    'lag_1': ticker_data_lag_1,
    'lag_2': ticker_data_lag_2,
    'lag_3': ticker_data_lag_3
}

print("Lead/lag data prepared!")

In [ ]:
# Lead-Lag Analysis
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

lead_lag_results = {}
for i, ticker in enumerate(STOCK_TICKERS):
    if ticker in ticker_data:
        indicator, zero_corr = analyze_lead_lag_correlations(ticker, axes[i], lead_lag_data)
        lead_lag_results[ticker] = {'indicator': indicator, 'zero_correlation': zero_corr}

plt.suptitle('Research Question 5: Lead/Lag Analysis')
plt.tight_layout()
plt.show()

print("\nLead/Lag Indicators (negative = sentiment lags prices):")
for ticker, result in lead_lag_results.items():
    print(f"{ticker}: {result['indicator']:.3f}")

print("\nConclusion Q5: WSB sentiment primarily follows rather than predicts price movements.")
print("Five of six tickers show negative indicators, confirming reactive behavior.")

## Summary of Results

### Key Findings:
1. **Discussion Consistency**: WSB discussion is highly volatile with dramatic spikes
2. **Volume Correlation**: Strong positive correlations (r=0.14-0.71) between mentions and trading volume
3. **Volatility Prediction**: Moderate correlations (r=0.21-0.46) between mentions and price volatility
4. **Sentiment Prediction**: Minimal predictive value (r≤0.12) for price direction
5. **Lead/Lag**: WSB primarily reacts to rather than predicts market movements

### Statistical Framework:
- Used Bonferroni correction for multiple testing (α = 0.0083)
- All volume and volatility correlations significant at p < 0.001
- Sentiment correlations weak despite some statistical significance